In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
import pandas as pd
import numpy as np
import pickle

from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, recall_score, precision_score

In [3]:
df_test = pd.read_csv(project.get_file('features_test.csv'),index_col=0)

scaler, LR_clfr, thresh = pickle.load(project.get_file('scaler_and_LR_clfr_and_thresh.pkl'))

In [4]:
num_recipes = len(df_test)
num_target = len(df_test[df_test['target'] == 1].index)

print('The test set contains ' + str(num_recipes) + ' recipes.')
print('The number of recipes which will achieve the popularity threshold by the target time is ' + str(num_target) + '.')
print('So we are trying to predict an outcome achieved by ' + str(100 * num_target / num_recipes) + '% of recipes.')

The test set contains 228553 recipes.
The number of recipes which will achieve the popularity threshold by the target time is 123.
So we are trying to predict an outcome achieved by 0.05381683898264297% of recipes.


In [5]:
X_test = df_test.drop(columns='target')
y_test = df_test['target']

X_test_scaled = scaler.transform(X_test)

predictions = LR_clfr.predict_proba(X_test_scaled)[:,1]

print('Model Scores.')
print('Area under the ROC curve: ' + str(roc_auc_score(y_test,predictions)))
print('Average precision score: ' + str(average_precision_score(y_test,predictions)))
print('Recall score: ' + str(recall_score(y_test,(predictions >= thresh))))
print('Precision score: ' + str(precision_score(y_test,(predictions >= thresh))))

Model Scores.
Area under the ROC curve: 0.9990094633249444
Average precision score: 0.35527220179913804
Recall score: 0.2764227642276423
Precision score: 0.5074626865671642


In [6]:
df_test_predictions = y_test.to_frame()
df_test_predictions['predicted_proba'] = predictions
df_test_predictions['prediction'] = (predictions >= thresh)

df_test_predictions.sort_values('predicted_proba', ascending=False)[:30]

,target,predicted_proba,prediction
recipe_id,,,
517863,0,0.188389,True
203785,1,0.156987,True
63746,0,0.142931,True
160202,1,0.142042,True
190605,1,0.139543,True
475446,0,0.139003,True
361631,1,0.136124,True
63073,0,0.134314,True
125120,0,0.133691,True


In [7]:
num_predictions = len(df_test_predictions[df_test_predictions['prediction'] == True])
num_correct_predictions = len(df_test_predictions[(df_test_predictions['prediction'] == True) & (df_test_predictions['target'] == 1)])

print('Looking at ' + str(num_recipes) + ' recipes, our model predicts that ' + str(num_predictions) + ' will achieve the popularity threshold by the target time.')
print()
print('Of these predictions, ' + str(num_correct_predictions) + ' of them are correct.')
print()
print('The model fails to predict ' + str(num_target - num_correct_predictions) + ' recipes which do achieve the threshold.')

Looking at 228553 recipes, our model predicts that 67 will achieve the popularity threshold by the target time.

Of these predictions, 34 of them are correct.

The model fails to predict 89 recipes which do achieve the threshold.


In [19]:
feature_weights = pd.DataFrame(LR_clfr.coef_[0],index=df_test.columns[1:]).rename(columns={0:'Weight'})
feature_weights.index.names = ['Feature']

feature_weights

,Weight
Feature,
total_ratings_curr,9.045011
age_in_days,-1.189304
rating_pace,0.284021
RUSNB_proba_pred,0.577928
mean_rating_curr,-0.989466
minutes,-0.000419
n_steps,0.603184
n_ingredients,0.343740
calories,0.002737


In [27]:
feature_weights['Absolute_Weight'] = feature_weights['Weight'].apply(np.abs)
feature_weights.sort_values(by='Absolute_Weight',ascending=False)

,Weight,Absolute_Weight
Feature,,
total_ratings_curr,9.045011,9.045011
age_in_days,-1.189304,1.189304
mean_rating_curr,-0.989466,0.989466
innovation_IDF,0.924896,0.924896
innovation_jaccard_avg,0.917794,0.917794
n_steps,0.603184,0.603184
RUSNB_proba_pred,0.577928,0.577928
LIX_score,-0.351900,0.351900
n_ingredients,0.343740,0.343740
